# Comparador baseado em regras

Notes: Nesta lição, vamos dar uma olhada no Comparador (Matcher) da biblioteca spaCy, que permite a criação de regras para encontrar palavras e frases no texto.

# Por que usar o Comparador e não somente expressões regulares?

- Permite a comparação com objetos `Doc` e não apenas texto (strings)
- Permite a comparação com os tokens e seus atributos
- Utiliza a previsão de um modelo
- Exemplo: "gentil" (adjetivo) vs. "gentil" (substantivo)

Notes: Além de comparar com texto (strings), que é o caso das expressões regulares, o Comparador (Matcher) também analisa os objetos `Doc` e `Token`.

Ele é bem mais flexível: você pode fazer a comparação no texto mas também nos seus atributos léxicos.

Você pode até criar regras que usam previsões de um modelo.

Por exemplo, você pode procurar a palavra "gentil"  somente se for um substantivo e não um adjetivo.


In [85]:
import spacy

nlp = spacy.load("pt_core_news_md")

In [86]:
doc = nlp("O nome do professor é Gentil")

In [87]:
for token in doc:
    # Imprimir o texto e a classe gramatical prevista
    print(token.text, token.pos_)

O DET
nome NOUN
do ADP
professor NOUN
é AUX
Gentil PROPN


In [16]:
doc = nlp("Como é gentil esse professor")

In [88]:
for token in doc:
    # Imprimir o texto e a classe gramatical prevista
    print(token.text, token.pos_)

O DET
nome NOUN
do ADP
professor NOUN
é AUX
Gentil PROPN


In [13]:
spacy.explain("PROPN")

'proper noun'

# Expressões de correspondência


- Listas de dicionários, uma por token
- Corresponde exatamente ao texto de um token:

In [19]:
[{"TEXT": "iPhone"}, {"TEXT": "X"}]

[{'TEXT': 'iPhone'}, {'TEXT': 'X'}]

- Corresponde a atributos léxicos:

In [20]:
[{"LOWER": "iphone"}, {"LOWER": "x"}]

[{'LOWER': 'iphone'}, {'LOWER': 'x'}]

- Corresponde a qualquer atributo de um token:

In [21]:
[{"LEMMA": "buy"}, {"POS": "NOUN"}]

[{'LEMMA': 'buy'}, {'POS': 'NOUN'}]

Notes: As expressões de correspondência são listas de dicionários. Cada dicionário se relaciona a um token. As chaves são os nomes dos atributos dos tokens, mapeadas para os valores esperados.

Neste exemplo, estamos procurando por dois tokens com o texto: "iPhone" e "X".

Podemos fazer a correspondência de acordo com outros atributos dos tokens. Neste exemplo estamos procurando dois tokens cuja forma em letras minúsculas corresponda a "iphone" e "x".

Podemos até escrever expressões usando atributos previstos por um modelo. Neste exemplo estamos procurando um token cujo lema seja "buy" seguido de um substantivo. O lema é o formato base da palavra.

In [89]:
import spacy

# Importar o Comparador (Matcher)
from spacy.matcher import Matcher

# Carregar o fluxo (pipeline) de processamento e criar um objeto nlp
nlp = spacy.load("pt_core_news_md")

# Inicializar o comparador com o vocabulário 
matcher = Matcher(nlp.vocab)

# Adicionar a expressão ao comparador
pattern = [{"TEXT": "iPhone"}, {"TEXT": "X"}]
matcher.add("IPHONE_PATTERN", [pattern])

# Processar um texto
doc = nlp("A data de lançamento do próximo iPhone X vazou")

# Chamar o matcher no doc
matches = matcher(doc)

In [90]:
matches

[(9528407286733565721, 6, 8)]

Notes: Para usar uma expressão, devemos importar o comparador `spacy.matcher`.

É necessário carregar um fluxo (pipeline) de processamento e criar um objeto `nlp`.

O comparador será inicializado com o vocabulário `nlp.vocab`. 

O método `matcher.add` permite adicionar uma expressão ao comparador. O primeiro argumento é o identificador único da expresssão que terá correspondência no texto. O segundo argumento é uma lista de expressões de correspondência.

Para fazer a correspondência de uma expressão em um texto, chame o comparador (matcher) e passe o texto como parâmetro.

Ele retornará as correspondências.

# Usando o Comparador (Matcher) (2)

In [27]:
# Chamar o comparador e passar o texto
doc = nlp("A data de lançamento do próximo iPhone X vazou")
matches = matcher(doc)

# Iterar nas correspondências
for match_id, start, end in matches:
    # Selecionar a partição que houve correspondência
    matched_span = doc[start:end]
    print(matched_span.text)

iPhone X


- `match_id`: código hash da expressão
- `start`: índice inicial da partição em que houve correspondência
- `end`: índice final da partição em que houve correspondência

Notes: Quando você usa o comparador em um documento (doc), ele retorna uma lista de tuplas.

Cada tupla consiste em três valores: o ID a expressão, o índice inicial e o índice final da partição em que houve correspondência.

Desta forma é possível iterar nas correspondências e criar um objeto partição `Span` : a parte do texto correspondente (do índice inicial até o índice final).



# Expressões com atributos léxicos

In [92]:
pattern = [
    {"LOWER": "copa"},
    {"LOWER": "do"},
    {"LOWER": "mundo"},
    {"LOWER": "fifa"},
    {"IS_DIGIT": True},
    {"IS_PUNCT": True}
]

In [93]:
doc = nlp("Copa do Mundo FIFA 2018: França venceu!")

In [94]:
matcher = Matcher(nlp.vocab)

# Adicionar a expressão ao comparador
matcher.add("FIFA_PATTERN", [pattern])

In [95]:
matches = matcher(doc)

In [98]:
matches

[(17311505950452258848, 0, 6)]

In [99]:
for match_id, start, end in matches:
    # Selecionar a partição que houve correspondência
    matched_span = doc[start:end]
    print(matched_span.text)

Copa do Mundo FIFA 2018:


# Expressões com outros atributos dos tokens

In [100]:
pattern = [
    {"LEMMA": "amar", "POS": "VERB"},
    {"POS": "NOUN"}
]

In [101]:
doc = nlp("Eu amava cachorros, mas agora passei a amar gatos também")

In [102]:
matcher = Matcher(nlp.vocab)
# Adicionar a expressão ao comparador
matcher.add("LOVE_PATTERN", [pattern])
matches = matcher(doc)
matches
for match_id, start, end in matches:
    # Selecionar a partição que houve correspondência
    matched_span = doc[start:end]
    print(matched_span.text)

amava cachorros
amar gatos


Note: Neste exemplo, estamos procurando por dois tokens:

Um verbo com o lema "amar", seguido de um substantivo.

Esta expressão terá correspondência com "amava cachorros" e "amar gatos".


# Utilizando operadores e quantificadores (1)

In [103]:
pattern = [
    {"LEMMA": "comprar"},
    {"POS": "DET", "OP": "?"},  # opcional: corresponde a 0 ou 1 vez
    {"POS": "NOUN"}
]

In [104]:
doc = nlp("estava Comprando um smartphone. Agora estou comprando aplicativos.")

In [105]:
matcher = Matcher(nlp.vocab)
# Adicionar a expressão ao comparador
matcher.add("LOVE_PATTERN", [pattern])
matches = matcher(doc)
matches
for match_id, start, end in matches:
    # Selecionar a partição que houve correspondência
    matched_span = doc[start:end]
    print(matched_span.text)

Comprando um smartphone
comprando aplicativos



Notes: Operadores e quantificadores permitem definir quantas vezes deverá haver correspondência com a expressão. Eles podem ser usados com a chave "OP".

Neste exemplo, o operador "?" faz com que a ocorrência seja opcional, então a expressão corresponderá a um token com o lema "comprar", um artigo (opcional) e um substantivo.



# Utilizando operadores e quantificadores (2)

| Exemplo       | Descrição                        |
| ------------- | ----------------------------     |
| `{"OP": "!"}` | Negação: corresponde 1 vez       |
| `{"OP": "?"}` | Opcional: corresponde 0 ou 1 vez |
| `{"OP": "+"}` | Corresponde 1 ou mais vezes      |
| `{"OP": "*"}` | Corresponde 1 ou mais vezes      |

Notes: "OP" pode ter um dos quatro valores abaixo:

"!" nega o valor do token, então corresponde a nenhuma ocorrência.

"?" faz o token opcional, corresponde a 0 ou 1 ocorrência.

"+" corresponde ao token uma ou mais ocorrências do token.

E "\*" corresponde a zero ou mais ocorrências do token.

Os operadores dão poder às suas expressões, mas por outro lado são mais complexos, use-os com sabedoria.